# Amazon Personalize

[Amazon Personalize](https://docs.aws.amazon.com/personalize/latest/dg/what-is-personalize.html) 是一个完全托管的机器学习服务，它使用您的数据为用户生成物品推荐。它还可以根据用户对特定物品或物品元数据的亲和度生成用户分群。

本笔记本介绍如何使用 Amazon Personalize Chain。在开始使用以下笔记本之前，您需要一个 Amazon Personalize campaign_arn 或 recommender_arn。

以下是一个在 Amazon Personalize 上设置 campaign_arn/recommender_arn 的[教程](https://github.com/aws-samples/retail-demo-store/blob/master/workshop/1-Personalization/Lab-1-Introduction-and-data-preparation.ipynb)。一旦设置好 campaign_arn/recommender_arn，您就可以在 langchain 生态系统中使用它。


## 1. 安装依赖

In [ ]:
!pip install boto3

## 2. 示例用例

### 2.1 [用例-1] 设置 Amazon Personalize 客户端并获取推荐

In [ ]:
from langchain_experimental.recommenders import AmazonPersonalize

recommender_arn = "<insert_arn>"

client = AmazonPersonalize(
    credentials_profile_name="default",
    region_name="us-west-2",
    recommender_arn=recommender_arn,
)
client.get_recommendations(user_id="1")

### 2.2 [用例-2] 调用 Personalize Chain 来汇总结果

In [ ]:
from langchain.llms.bedrock import Bedrock
from langchain_experimental.recommenders import AmazonPersonalizeChain

bedrock_llm = Bedrock(model_id="anthropic.claude-v2", region_name="us-west-2")

# Create personalize chain
# Use return_direct=True if you do not want summary
chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False
)
response = chain({"user_id": "1"})
print(response)

### 2.3 [用例-3] 使用自定义提示调用 Amazon Personalize Chain

In [ ]:
from langchain.prompts.prompt import PromptTemplate

RANDOM_PROMPT_QUERY = """
You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, 
    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. 
    The movies to recommend and their information is contained in the <movie> tag. 
    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. 
    Put the email between <email> tags.

    <movie>
    {result} 
    </movie>

    Assistant:
    """

RANDOM_PROMPT = PromptTemplate(input_variables=["result"], template=RANDOM_PROMPT_QUERY)

chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False, prompt_template=RANDOM_PROMPT
)
chain.run({"user_id": "1", "item_id": "234"})

### 2.4 [用例-4] 在顺序链中调用 Amazon Personalize

In [ ]:
from langchain.chains import LLMChain, SequentialChain

RANDOM_PROMPT_QUERY_2 = """
You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, 
    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. 
    You want the email to impress the user, so make it appealing to them.
    The movies to recommend and their information is contained in the <movie> tag. 
    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. 
    Put the email between <email> tags.

    <movie>
    {result}
    </movie>

    Assistant:
    """

RANDOM_PROMPT_2 = PromptTemplate(
    input_variables=["result"], template=RANDOM_PROMPT_QUERY_2
)
personalize_chain_instance = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=True
)
random_chain_instance = LLMChain(llm=bedrock_llm, prompt=RANDOM_PROMPT_2)
overall_chain = SequentialChain(
    chains=[personalize_chain_instance, random_chain_instance],
    input_variables=["user_id"],
    verbose=True,
)
overall_chain.run({"user_id": "1", "item_id": "234"})

### 2.5 [用例-5] 调用 Amazon Personalize 并检索元数据

In [ ]:
recommender_arn = "<insert_arn>"
metadata_column_names = [
    "<insert metadataColumnName-1>",
    "<insert metadataColumnName-2>",
]
metadataMap = {"ITEMS": metadata_column_names}

client = AmazonPersonalize(
    credentials_profile_name="default",
    region_name="us-west-2",
    recommender_arn=recommender_arn,
)
client.get_recommendations(user_id="1", metadataColumns=metadataMap)

### 2.6 [用例-6] 使用返回的元数据调用 Personalize Chain 来汇总结果

In [ ]:
bedrock_llm = Bedrock(model_id="anthropic.claude-v2", region_name="us-west-2")

# Create personalize chain
# Use return_direct=True if you do not want summary
chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False
)
response = chain({"user_id": "1", "metadata_columns": metadataMap})
print(response)